In [ ]:
from  PIL import Image
import PIL.Image
import redis
import PIL
import os
import cv2
# from module import k_mean_color_detection
import numpy as np
import random
from module import  process_raw_d_logs, process_raw_cc_logs, process_cc_logs

# Connect to the Redis server
r = redis.Redis(host='localhost', port=6379, db=0)

def clear_redis_database():
    r.flushdb()
    print("Redis database cleared.")

# Function to store string data with key namespacing
def store_string_with_namespace(serial_number, constant_string, value):
    full_key = f"{serial_number}:{constant_string}"
    r.set(full_key, value)
    print(f"Stored {full_key} -> {value}")

# Function to retrieve all data matching the constant part of the key
def retrieve_keys(constant_string):
    matching_keys = r.keys(f"*:{constant_string}")
    results = {}
    for key in matching_keys:
        value = r.get(key)
        if value:
            decoded_key = key.decode()
            decoded_value = value.decode()
            # np_arr = np.frombuffer(value, np.uint8)
            # Decode the numpy array back to an image
            # retrieved_image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
            # results[decoded_key] = retrieved_image
            results[decoded_key] = decoded_value
            # r.delete(decoded_key)
            # print(results)
            # r.delete(key)  # Delete the key after retrieving the value
            # print(f"Retrieved and deleted {decoded_key} -> {decoded_value}")
    return results

def retrieve_image(constant_string):
    matching_keys = r.keys(f"*:{constant_string}")
    results = {}
    for key in matching_keys:
        value = r.get(key)
        if value:
            decoded_key = key.decode()
            # decoded_value = value.decode()
            np_arr = np.frombuffer(value, np.uint8)
            # Decode the numpy array back to an image
            image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
            # image = image.reshape((image.shape[0] * image.shape[1], 3))
            # image = image.reshape((-1, image.shape[2]))
            # cv2.imwrite(f"/home/annone/ai/backend/stream/d_temp/{decoded_key}.png", image)
            results[decoded_key] = image
            # results[decoded_key] = decoded_value
            # r.delete(decoded_key)
            # print(results)
            # r.delete(key)  # Delete the key after retrieving the value
            # print(f"Retrieved and deleted {decoded_key} -> {decoded_value}")
    return results

# def process_raw_d_logs(constant_string):
#     while True:
#         matching_keys = r.keys(f"*:{constant_string}")
#         # print(matching_keys)
#         for key in matching_keys:
#             value = r.get(key)
#             decode_key = key.decode()
#             decode_value = value.decode().split("|")
#             track_id = decode_value[5]
#             image_path = decode_value[7]
#             try:
#                 image = cv2.imread(f"/home/annone/ai-camera/backend/stream/temp/{image_path}")
#                 image = image.reshape((image.shape[0] * image.shape[1], 3))
#                 aa = k_mean_color_detection(image)
#                 decode_value[7]  = aa
#                 r.set(f"{track_id}_{random.randint(0,9999)}:d_log","|".join(decode_value))
#                 os.remove(f"/home/annone/ai-camera/backend/stream/temp/{image_path}")
#                 r.delete(decode_key)
#             except:
#                 # print("excepted")
#                 continue
            # show_rgb_colors(aa,image)
            # print(decode_value)

# def process_d_logs(constant_string):
#     conn = Database.get_connection()
#     cursor = conn.cursor()
#     batch = []
#     while True:
#         matching_keys = r.keys(f"*:{constant_string}")
#         for key in matching_keys:
#             value = r.get(key)
#             decode_key = key.decode()
#             decode_value = value.decode().split("|")
#             batch.append(tuple(decode_value))
#         if len(batch) > 0:
#             query = 'INSERT INTO "DetectionLog" ("cameraId", "camera_ip", "timestamp", "boxCoords", "detectionClass", "trackId", "classConfidence", "metadata") VALUES %s;'
#             execute_values(cursor, query, batch)
#             conn.commit()
#             cursor.close()
#             conn.close()
#             batch = []

In [ ]:
retrieve_image("image")["817_qcYXs5PM_39_1724913805_127_0_0_1_976:image"]

In [ ]:
retrieve_keys("raw_d_log")

In [ ]:
clear_redis_database()

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import redis

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, db=0)

# Get all keys from Redis
keys = r.keys('*')

# Loop through the keys and get the data for each key
for i, key in enumerate(keys):
    if i ==10:
        break
    # Decode key to get human-readable format
    decoded_key = key.decode('utf-8')
    
    # Get the value associated with the key
    value = r.get(key)
    
    # Decode the value, if it's stored as a string
    decoded_value = value.decode('utf-8') if value else None
    
    # Print the key-value pair
    print(f"Key: {decoded_key}, Value: {decoded_value}")


## Boundig Box

In [ ]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from sort.sort import Sort  # Example of an external tracking library
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO("/home/annone/ai/models/objseg50e.pt")
model.to(device)
print(device)

tracker = Sort()

# Function to process frames in batches
def process_frame_batch(frames):
    resized_frames = [cv2.resize(frame, (640 // 32 * 32, 480 // 32 * 32)) for frame in frames]

    frames_tensor = torch.from_numpy(np.stack(resized_frames)).permute(0, 3, 1, 2).float().to(device) / 255.0

    with torch.no_grad():
        batch_results = model(frames_tensor, device=device)

    return batch_results, resized_frames

def track_objects(frames, batch_results):
    tracked_frames = []
    
    for frame, result in zip(frames, batch_results):
        detections = []
        
        if result.boxes:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                score = box.conf[0].item()
                label = model.names[int(box.cls[0])]
                
                detections.append([x1, y1, x2, y2, score])

        tracks = tracker.update(np.array(detections))

        for track in tracks:
            track_id = int(track[4])
            x1, y1, x2, y2 = map(int, track[:4])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        tracked_frames.append(frame)
    
    return tracked_frames

def stream_process(camera_id, camera_ip, video_path, batch_size=8):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    frames = []
    t1 = time.time()
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frames.append(frame)

        if len(frames) >= batch_size:
            batch_results, resized_frames = process_frame_batch(frames)

            tracked_frames = track_objects(resized_frames, batch_results)

            for tracked_frame in tracked_frames:
                cv2.imshow("Tracked Frame", tracked_frame)
            frames = []
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    t2 = time.time()
    print(t2-t1)
    cap.release()
    cv2.destroyAllWindows()

video_path = '/home/annone/ai/data/T-pole wrong way.mp4'
cam_ip = '127.0.0.1'
cam_id = "1"
stream_process(cam_id, cam_ip, video_path, batch_size=2)


## Segmention

In [ ]:
offset=10
count = 0
total_parking_violations = 0 
wrong_way_violation_count = 0
traffic_violation_count = 0  
crossed_objects = {}  # Track objects that have crossed lines
violated_objects = set()  # Track objects that have already violated
static_objects = {}  # To track objects that are stationary
stationary_frame_threshold = 200
# parking
def check_illegal_parking(track_id, cx, cy):
    """Check if an object is illegally parked based on stationary duration."""
    global static_objects, total_parking_violations

    # Initialize tracking information if not already present
    if track_id not in static_objects:
        static_objects[track_id] = {"position": (cx, cy), "frames": 0, "violated": False}
    else:
        last_position = static_objects[track_id]["position"]

        # Check if the object has remained stationary (within a certain offset)
        if abs(cx - last_position[0]) <= offset and abs(cy - last_position[1]) <= offset:
            static_objects[track_id]["frames"] += 1
            # If the object is stationary for more than the threshold and not already marked as a violation
            if static_objects[track_id]["frames"] > stationary_frame_threshold and not static_objects[track_id]["violated"]:
                static_objects[track_id]["violated"] = True  # Mark the object as a violation
                total_parking_violations += 1  # Increase the parking violation count
                print(f"Object {track_id} marked as parking violation. Total Violations: {total_parking_violations}")
        else:
            # If the object has moved, reset its tracking information
            static_objects[track_id]["position"] = (cx, cy)
            static_objects[track_id]["frames"] = 0
            static_objects[track_id]["violated"] = False
# wrong way
def detect_wrong_way_violation(track_id, cx, cy):
    global wrong_way_violation_count, violated_objects

    # Initialize tracking for the object if not already done
    if track_id not in crossed_objects:
        crossed_objects[track_id] = {'red': set(), 'green': set()}

    # Check if the object crosses any red line
    for i, ((x_start, y_start), (x_end, y_end)) in enumerate(red_lines):
        if min(y_start, y_end) - offset <= cy <= max(y_start, y_end) + offset:
            if min(x_start, x_end) <= cx <= max(x_start, x_end):
                crossed_objects[track_id]['red'].add(i)

    # Check if the object crosses any green line
    for i, ((x_start, y_start), (x_end, y_end)) in enumerate(green_lines):
        if min(y_start, y_end) - offset <= cy <= max(y_start, y_end) + offset:
            if min(x_start, x_end) <= cx <= max(x_start, x_end):
                crossed_objects[track_id]['green'].add(i)

    # Detect wrong-way violation (crossing green line after red line)
    if any(
        i in crossed_objects[track_id]['green'] and
        i in crossed_objects[track_id]['red'] and
        track_id not in violated_objects
        for i in crossed_objects[track_id]['green']
    ):
        # cv2.putText(frame, "Wrong Way Violation", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        wrong_way_violation_count += 1
        violated_objects.add(track_id)
# red light violation
def detect_traffic_violation(track_id, cx, cy):
    """Check if an object violates traffic rules by crossing lines."""
    global traffic_violation_count, crossed_objects, violated_objects

    # Initialize tracking for the object if not already done
    if track_id not in crossed_objects:
        crossed_objects[track_id] = set()

    # Check if the object crosses any red line
    for i, ((x_start, y_start), (x_end, y_end)) in enumerate(red_lines):
        if min(y_start, y_end) - offset <= cy <= max(y_start, y_end) + offset:
            if min(x_start, x_end) <= cx <= max(x_start, x_end):
                crossed_objects[track_id].add(f"red_{i}")

    # Check if the object crosses any green line after crossing a red line (indicating a violation)
    for i, ((x_start, y_start), (x_end, y_end)) in enumerate(green_lines):
        if min(y_start, y_end) - offset <= cy <= max(y_start, y_end) + offset:
            if min(x_start, x_end) <= cx <= max(x_start, x_end):
                if any(f"red_{j}" in crossed_objects[track_id] for j in range(len(red_lines))) and track_id not in violated_objects:
                    # cv2.putText(frame, "Traffic Violation", (cx, cy - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                    traffic_violation_count += 1
                    violated_objects.add(track_id)

In [6]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from sort.sort import Sort
import time
# from module import generate_custom_string
from collections import defaultdict
import math
import redis
import uuid
import subprocess

# Initialize device and model
PARENT_DIR = "/home/annone/ai"
r = redis.Redis(host='localhost', port=6379, db=0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO("/home/annone/ai/models/objseg50e.pt")
model.to(device)
print(f"{device} as Computation Device initiated")
tracker = Sort()

# CONSTANTS PER CAMERA
width, height = 640,480
camera_ip = "198.78.45.89"
camera_id = 2
fps = 30
class_list = ['auto', 'bike-rider', 'bolero', 'bus', 'car', 'hatchback', 'jcb', 'motorbike-rider', 'omni', 'pickup',
              'scooty-rider', 'scorpio', 'sedan', 'suv', 'swift', 'thar', 'tractor', 'truck', 'van']
previous_positions = defaultdict(lambda: {"x": 0, "y": 0, "time": 0})
null_mask = np.zeros((height, width), dtype=np.uint8)

track_ids_inframe = {}
custom_track_ids = {}
known_track_ids = []


# STREAMING CONSTANTS
rtsp_url = 'rtsp://localhost:8554/stream1'  # Update this URL as needed

# Construct the FFmpeg command
ffmpeg_cmd = [
    'ffmpeg',
    '-y',  # Overwrite output files without asking
    '-f', 'rawvideo',  # Input format
    '-pix_fmt', 'bgr24',  # Pixel format
    '-s', '640x480',  # Video resolution (adjust as needed)
    '-r', '25',  # Frame rate
    '-i', '-',  # Input from stdin
    '-c:v', 'libx264',  # Video codec
    '-preset', 'ultrafast',  # Encoding speed
    '-tune', 'zerolatency',  # Tune for low latency
    '-f', 'rtsp',  # Output format
    rtsp_url  # RTSP output URL
]
process = subprocess.Popen(ffmpeg_cmd, stdin=subprocess.PIPE)


# Function to calculate distance in pixels
def calculate_pixel_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Function to calculate speed (assuming pixel distance and time interval)
def calculate_speed(pixel_distance, time_interval):
    return pixel_distance / time_interval  # Speed in pixels per second

# Function to process frames in batches
def process_frame_batch(frames):
    resized_frames = [cv2.resize(frame, (640 // 32 * 32, 480 // 32 * 32)) for frame in frames]
    frames_tensor = torch.from_numpy(np.stack(resized_frames)).permute(0, 3, 1, 2).float().to(device) / 255.0

    with torch.no_grad():
        batch_results = model(frames_tensor, device=device)

    return batch_results, resized_frames


# Function to generate a custom track ID based on YOLO class, confidence, and a unique UUID
def generate_custom_track_id(label, confidence):
    return f"{label}_{confidence:.2f}_{uuid.uuid4()}"

# Function to track objects and draw segmentation polygons
def track_objects(frames, batch_results, frame_time):
    global camera_ip, previous_positions, fps, camera_id, track_ids_inframe, custom_track_ids, known_track_ids

    tracked_frames = []
    current_track_ids = []  # To keep track of the tracks currently in the frame

    for frame, result in zip(frames, batch_results):
        detections = []
        img_bin = []
        labels = []
        confs = []

        if result.masks:
            for mask, box in zip(result.masks.xy, result.boxes):
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                score = box.conf[0].item()
                label = model.names[int(box.cls[0])]
                detections.append([x1, y1, x2, y2, score])
                labels.append(label)
                confs.append(score)
                cv2.polylines(frame, [np.array(mask, dtype=np.int32)], isClosed=True, color=(0, 255, 0), thickness=1)
                cv2.putText(frame, f"{label} ({score:.2f})", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

        tracks = tracker.update(np.array(detections))

        for i, track in enumerate(tracks):
            track_id = int(track[4])
            x1, y1, x2, y2 = map(int, track[:4])

            # If the object is new, generate a custom track ID and store initial data
            if track_id not in custom_track_ids:
                custom_id = generate_custom_track_id(labels[i], confs[i])
                custom_track_ids[track_id] = {
                    "custom_track_id": custom_id,
                    "camera_id": camera_id,
                    "camera_ip": camera_ip,
                    "first_appearance": frame_time,  # Store first appearance time
                    "last_appearance": frame_time,   # Initialize last appearance time
                    "dbbox": [[x1, y1, x2, y2]],
                    "dlabel": [labels[i]],
                    "dconf": [confs[i]],
                }
            else:
                # Append the new frame data to the existing object in the dict
                custom_track_ids[track_id]["dbbox"].append([x1, y1, x2, y2])
                custom_track_ids[track_id]["dlabel"].append(labels[i])
                custom_track_ids[track_id]["dconf"].append(confs[i])
                custom_track_ids[track_id]["last_appearance"] = frame_time  # Update last appearance time

            # Add current track ID to the list of track IDs in the current frame
            current_track_ids.append(track_id)

            # Display the custom track ID on the frame
            cv2.putText(frame, f"ID: {custom_track_ids[track_id]['custom_track_id']}", (x1, y1 - 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)

            # Append the current frame to the tracked frames
            tracked_frames.append(frame)

    # Check for tracks that are no longer in the current frame (left the frame)
    tracks_left_frame = set(custom_track_ids.keys()) - set(current_track_ids)

    # Insert data into Redis for tracks that left the frame
    for track_id in tracks_left_frame:
        track_data = custom_track_ids[track_id]
        # r.set(track_data['custom_track_id'], str(track_data))  # Insert into Redis as a string or JSON

        # Remove the track ID from the custom_track_ids since it left the frame
        del custom_track_ids[track_id]

    return tracked_frames, list(custom_track_ids.keys())


# Function to stream video and process frames in batches
def stream_process(camera_id, camera_ip, video_path, batch_size=8):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter("/home/annone/ai/data/output.mp4", fourcc, fps, (640,480))
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    frames = []
    t1 = time.time()
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Record the current time in seconds for tracking purposes
        frame_time = time.time()

        frames.append(frame)

        if len(frames) >= batch_size:
            batch_results, resized_frames = process_frame_batch(frames)

            tracked_frames, track_id_list = track_objects(resized_frames, batch_results, frame_time)
            for tracked_frame in tracked_frames:
                cv2.imshow("Tracked Frame", tracked_frame)
                process.stdin.write(tracked_frame.tobytes())
                out.write(tracked_frame)
            frames = []
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    t2 = time.time()
    print(t2-t1)
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(custom_track_ids)

# Example usage
video_path = '/home/annone/ai/data/test.mp4'
cam_ip = '127.0.0.1'
cam_id = "1"
stream_process(cam_id, cam_ip, video_path, batch_size=2)


cuda as Computation Device initiated

0: 480x640 1 auto, 1 helmet, 5 motorbike-riders, 1 scooty-rider, 5.6ms
1: 480x640 1 auto, 4 motorbike-riders, 1 scooty-rider, 1 tractor, 5.6ms
Speed: 0.0ms preprocess, 5.6ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)


ffmpeg version 4.4.5-0ubuntu1~22.04.sav0 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.4.0-1ubuntu1~22.04)
  configuration: --prefix=/usr --extra-version='0ubuntu1~22.04.sav0' --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-li


0: 480x640 1 auto, 4 motorbike-riders, 1 scooty-rider, 1 tractor, 5.2ms
1: 480x640 1 auto, 4 motorbike-riders, 1 scooty-rider, 1 tractor, 5.2ms
Speed: 0.0ms preprocess, 5.2ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)


Input #0, rawvideo, from 'pipe:':
  Duration: N/A, start: 0.000000, bitrate: 184320 kb/s
  Stream #0:0: Video: rawvideo (BGR[24] / 0x18524742), bgr24, 640x480, 184320 kb/s, 25 tbr, 25 tbn, 25 tbc
Stream mapping:
  Stream #0:0 -> #0:0 (rawvideo (native) -> h264 (libx264))
[libx264 @ 0x5be0f4501940] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x5be0f4501940] profile High 4:4:4 Predictive, level 3.0, 4:4:4, 8-bit
[libx264 @ 0x5be0f4501940] 264 - core 164 r3191 4613ac3 - H.264/MPEG-4 AVC codec - Copyleft 2003-2024 - http://www.videolan.org/x264.html - options: cabac=0 ref=1 deblock=0:0:0 analyse=0:0 me=dia subme=0 psy=1 psy_rd=1.00:0.00 mixed_ref=0 me_range=16 chroma_me=1 trellis=0 8x8dct=0 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=6 threads=7 lookahead_threads=7 sliced_threads=1 slices=7 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=0 weightp=0 keyint=250 keyint_min=25 scenecut=0 intra_refresh=0 rc=crf mbtree=0 crf=2


0: 480x640 1 auto, 5 motorbike-riders, 1 scooty-rider, 1 tractor, 5.3ms
1: 480x640 2 autos, 5 motorbike-riders, 1 scooty-rider, 1 tractor, 5.3ms
Speed: 0.0ms preprocess, 5.3ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 auto, 5 motorbike-riders, 5.1ms
1: 480x640 1 auto, 1 man, 3 motorbike-riders, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 auto, 1 man, 2 motorbike-riders, 5.1ms
1: 480x640 2 autos, 1 man, 3 motorbike-riders, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 1 man, 5 motorbike-riders, 5.1ms
1: 480x640 1 auto, 1 bike-rider, 1 man, 4 motorbike-riders, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 1 bike-rider, 1 man, 4 motorbike-riders, 5.1ms
1: 480x640 3 autos, 4 motorbike-riders, 5.1ms
Speed: 0.0ms preprocess, 5.1ms i

...


0: 480x640 2 autos, 2 motorbike-riders, 1 no-helmet, 1 tractor, 1 truck, 5.2ms
1: 480x640 2 autos, 2 helmets, 2 motorbike-riders, 1 no-helmet, 1 tractor, 1 truck, 5.2ms
Speed: 0.0ms preprocess, 5.2ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 1 helmet, 2 motorbike-riders, 1 no-helmet, 1 truck, 5.1ms
1: 480x640 2 autos, 1 helmet, 2 motorbike-riders, 1 no-helmet, 1 tractor, 1 truck, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 1 helmet, 2 motorbike-riders, 1 no-helmet, 1 tractor, 1 truck, 5.1ms
1: 480x640 2 autos, 2 motorbike-riders, 4 no-helmets, 1 tractor, 1 truck, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 2 motorbike-riders, 1 no-helmet, 1 tractor, 1 truck, 5.1ms
1: 480x640 2 autos, 2 motorbike-riders, 2 no-helmets, 1 tractor, 1 truck, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.


0: 480x640 2 autos, 2 motorbike-riders, 1 no-helmet, 1 tractor, 1 truck, 5.1ms
1: 480x640 2 autos, 1 car, 1 helmet, 4 motorbike-riders, 3 no-helmets, 1 tractor, 1 truck, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 2 helmets, 3 motorbike-riders, 2 no-helmets, 1 truck, 5.1ms
1: 480x640 2 autos, 1 helmet, 3 motorbike-riders, 2 no-helmets, 1 truck, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 1 helmet, 2 motorbike-riders, 1 no-helmet, 1 truck, 5.1ms
1: 480x640 2 autos, 2 helmets, 3 motorbike-riders, 1 no-helmet, 1 truck, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 1 helmet, 2 motorbike-riders, 1 no-helmet, 1 truck, 5.1ms
1: 480x640 2 autos, 2 motorbike-riders, 1 no-helmet, 1 truck, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 2.0ms postprocess per image 


0: 480x640 2 autos, 1 helmet, 2 motorbike-riders, 1 truck, 5.1ms
1: 480x640 2 autos, 1 helmet, 2 motorbike-riders, 1 truck, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 1 helmet, 1 motorbike-rider, 1 truck, 5.2ms
1: 480x640 2 autos, 1 helmet, 1 motorbike-rider, 1 tractor, 1 truck, 5.2ms
Speed: 0.0ms preprocess, 5.2ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 1 helmet, 1 motorbike-rider, 1 truck, 5.1ms
1: 480x640 2 autos, 1 helmet, 1 motorbike-rider, 1 tractor, 1 truck, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 1 helmet, 1 motorbike-rider, 1 no-helmet, 1 truck, 5.2ms
1: 480x640 2 autos, 1 helmet, 1 motorbike-rider, 1 truck, 5.2ms
Speed: 0.0ms preprocess, 5.2ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 1 motorbike-rider, 1 no-helmet, 1 truck, 5


0: 480x640 2 autos, 1 helmet, 1 motorbike-rider, 1 tractor, 1 truck, 5.1ms
1: 480x640 2 autos, 1 helmet, 2 motorbike-riders, 1 no-helmet, 1 tractor, 1 truck, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 1 helmet, 1 license-plate, 1 motorbike-rider, 3 no-helmets, 1 tractor, 1 truck, 5.1ms
1: 480x640 2 autos, 1 helmet, 1 motorbike-rider, 2 no-helmets, 1 truck, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 1 helmet, 2 motorbike-riders, 3 no-helmets, 1 truck, 5.1ms
1: 480x640 2 autos, 1 helmet, 2 motorbike-riders, 2 no-helmets, 1 truck, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 1 motorbike-rider, 1 no-helmet, 1 tractor, 1 truck, 5.1ms
1: 480x640 2 autos, 1 helmet, 2 motorbike-riders, 2 no-helmets, 1 truck, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.8


0: 480x640 2 autos, 2 motorbike-riders, 1 truck, 5.2ms
1: 480x640 2 autos, 2 motorbike-riders, 1 truck, 5.2ms
Speed: 0.0ms preprocess, 5.2ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 2 motorbike-riders, 1 truck, 5.2ms
1: 480x640 2 autos, 2 motorbike-riders, 1 truck, 5.2ms
Speed: 0.0ms preprocess, 5.2ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 autos, 2 motorbike-riders, 1 truck, 5.1ms
1: 480x640 2 autos, 2 motorbike-riders, 1 truck, 5.1ms
Speed: 0.1ms preprocess, 5.1ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)
64.58660054206848
{4743: {'custom_track_id': 'motorbike-rider_0.96_f3df7c8c-3ddb-4dc4-8231-8a560d98dea9', 'camera_id': 2, 'camera_ip': '198.78.45.89', 'first_appearance': 1727070977.208253, 'last_appearance': 1727070986.4486728, 'dbbox': [[56, 119, 120, 207], [56, 120, 121, 207], [56, 120, 121, 206], [56, 120, 121, 206], [56, 120, 120, 206], [56, 120, 120, 205], [57, 120, 120,

In [ ]:
from collections import Counter, defaultdict

# 1. Mode-Based Approach: Top 3 most frequent labels
def top_3_label_mode(dlabel):
    label_counts = Counter(dlabel)
    top_3_labels = label_counts.most_common(3)
    return top_3_labels  # Returns label with its count

# 2. Weighted Confidence Sum Approach: Top 3 labels by cumulative confidence
def top_3_label_weighted_conf(dlabel, dconf):
    weighted_conf_dict = defaultdict(float)
    
    # Sum the confidences for each label
    for label, conf in zip(dlabel, dconf):
        weighted_conf_dict[label] += conf
    
    # Get top 3 labels with highest weighted confidence
    top_3_weighted = sorted(weighted_conf_dict.items(), key=lambda x: x[1], reverse=True)[:3]
    
    return top_3_weighted  # Returns label with its weighted confidence

# 3. Highest Confidence Approach: Top 3 labels with the highest individual confidence scores
def top_3_label_highest_conf(dlabel, dconf):
    # Combine labels and confidences into pairs and sort by confidence
    label_conf_pairs = sorted(zip(dlabel, dconf), key=lambda x: x[1], reverse=True)
    
    # Get top 3 based on confidence
    top_3_highest_conf = label_conf_pairs[:3]
    
    return top_3_highest_conf  # Returns label with its confidence

# Example usage
dlabel = ['tractor', 'hatchback', 'car', 'car', 'car', 'car', 'car', 'pickup', 'pickup', 'auto', 'auto', 'car', 
          'car', 'auto', 'auto', 'pickup', 'pickup', 'pickup', 'tractor', 'auto', 'auto', 'tractor', 'auto', 
          'tractor', 'car', 'car', 'tractor', 'tractor', 'tractor', 'pickup', 'motorbike-rider', 'pickup', 'car', 
          'tractor', 'car', 'car', 'auto']
dconf = [0.275, 0.274, 0.271, 0.333, 0.334, 0.443, 0.444, 0.392, 0.381, 0.429, 0.342, 0.356, 
         0.349, 0.500, 0.498, 0.387, 0.372, 0.463, 0.341, 0.618, 0.274, 0.350, 0.341, 0.331, 
         0.300, 0.279, 0.303, 0.391, 0.375, 0.465, 0.360, 0.466, 0.460, 0.434, 0.425, 0.482, 0.408]

# Get top 3 selections using different methods
top_3_mode = top_3_label_mode(dlabel)
top_3_weighted = top_3_label_weighted_conf(dlabel, dconf)
top_3_high_conf = top_3_label_highest_conf(dlabel, dconf)

# Print results
print("Top 3 labels (Mode-based):", top_3_mode)
print("Top 3 labels (Weighted confidence):", top_3_weighted)
print("Top 3 labels (Highest individual confidence):", top_3_high_conf)
